# Traffic Sign Classifier Project

In this example we will go through several paths of building ConvNets in Tensorflow 2.0 to build a traffic sign classifier.

This traffic sign classifier will use Deep Neural Networks, we will start by using a custom shallow neural network to test the classifier.

On the second stage we will build the same neural network using a dropout layer and finally in the third step we will build a neural network based on transfer learning.

Finally, after comparing the model we will see which one is the best by evaluating the metrics result.  Finally after the notebook is completed we will deploy the model into a heroku app.

### We will first import the necessary libraries

In [ ]:
from tensorflow.keras.layers.experimental.preprocessing import RandomTranslation, RandomContrast
from tensorflow.keras.layers.experimental.preprocessing import RandomRotation, RandomZoom 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import MobileNetV2, Xception, ResNet50V2
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.metrics import classification_report
from keras.utils.vis_utils import plot_model
from sklearn.metrics import confusion_matrix
from tensorflow.keras.utils import load_img
from tensorflow.keras.layers import Input
from tensorflow.keras import Sequential
from tensorflow.keras import Model
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from numpy import asarray
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib
import itertools
import warnings
import os

In [ ]:
%matplotlib inline
warnings.filterwarnings('ignore')

### Then we will specify the folders that will be needed in the process

Train will contain the images that will be used to modelate the classifier
Test will be used to demonstrate the performance of the classifier

All other csv files contains general information of the dataset, this dataset is labeled to be used as a benchmark for image classification.  The files contain the region of interest in format *(w, h, x1, y1, x2, y2, classId, image_path)*. Because we will be using the images in the classification pipeline, we do not have the need to use more than the *classId* and the *image_path*.

In [ ]:
# data paths
train_path    = 'Train'
test_path     = 'Test'

# data frames
training_data = 'Train.csv'
testing_data  = 'Test.csv'
label_names   = 'signnames.csv'

Now we will explore the datasets mentioned on the previous markdown

In [ ]:
train_df = pd.read_csv(training_data)
test_df = pd.read_csv(testing_data)
class_df = pd.read_csv(label_names)

As you will see in the next step we will only need to pop out the classId and the path of the image.

In [ ]:
train_df.head()

We will have now general insights of the statistics of the dataset

In [ ]:
train_df.describe()

Our dataset has basically 39209 labeled images, all classes are from 0-42 corresponding to 43 labels and near 50% of the dataset is the label 12.  Just for curiosity, let's check the test dataframe

In [ ]:
test_df.head()

In [ ]:
class_df.head()

As you saw above, we have a very good label for each target, so no need to worry about it, we can get out the name of the label give the value or location of the sign name.

For example, we could see that the last column is the name of the sign name related to the class Id, but we could follow the index of the dataframe to evaluate the classId

### Visual Insights

We will build a simple insight of the dataset for check if the dataset is imbalanced or not.

In [ ]:
matplotlib.style.use('ggplot') # change graph style
plt.figure(figsize=(8,8))      # change figure size

classes = train_df['ClassId']  # get the classes of the dataframe
res = classes.value_counts().sort_index()
res.plot(kind='bar', alpha=0.75, rot=90);

So our dataset is imbalanced, no problem.  There are methods for combat imbalanced datasets but currently we will not cover that part, we will only focus on the task of classification.

Below you will se some hyperparameters like the image batch sizes and the height and width properties.


In [ ]:
seed = 42                        # random seed to maintain the results
validation_split = 0.2           # percentage of the validation dataset
np.random.seed(seed)             # maitain same results
batch_size = 64                  # we will process each 64 image of dataset time by time until you complete the full dataset
img_width, img_height = 128, 128 # the size of the image

In [ ]:
# load the training dataset 
train_ds = image_dataset_from_directory(train_path,
                                       validation_split=validation_split,
                                       subset='training',
                                       seed=seed,
                                       image_size=(img_width, img_height),
                                       batch_size=batch_size)

In [ ]:
# load the validation dataset
val_ds = image_dataset_from_directory(train_path,
                                       validation_split=validation_split,
                                       subset='validation',
                                       seed=42,
                                       image_size=(img_width, img_height),
                                       batch_size=batch_size)

Another way to load images is using the *ImageDataGenerator()*  An example is below:
    
    train_datagen = ImageDataGenerator(rescale=1./255, shear_angle=0.2, zoom_range=0.2, horizontal_flip=False)

One thing to note is that all the transformations will be done to the training but not for the test set.
The other thing is that we could not us the horizontal flip because probably are traffic signals to move in one direction or another.

    test_datagen = ImageDataGenerator(rescale=1./255)
    
Now we can use the data generator as follows:

    train_generator = train_datagen.flow_from_directory(
        'Train',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')
        
    train_generator = test_datagen.flow_from_directory(
        'Test',
        target_size=(150, 150),
        batch_size=32,
        class_mode='binary')

### Exploratory Visualization of the Dataset

Now we will use the basics of matplotlib to explore the dataset.  Please take in consideration that *train_ds* and *valid_ds* have to outputs, of tf.data.Dataset and it has two tensors, the first one is for the images and the second one is for the labels.

Before starting, for easy understanding, we must match the output of the dataset with the output of the ground truth values, just formatting only, if you print the first lines of the *train_ds.class_names* and *class_names* dict you will find why doesn't match.

In [ ]:
train_ds.class_names[:10]

In [ ]:
[v for v in class_df['ClassId']][:10]

In [ ]:
class_df['ClassId'] = np.array(train_ds.class_names,dtype='int')
class_df

The first thing to note is the type, it is a string and we have integers.  The second one is the name also doesn't match, cases 0 and 1 are ok, but when it comes to 2, 3 and so on for class_names dictionary it will fail to match.  To eliminate this problem and cause confusion we will rematch the dictionary

In [ ]:
def visualize_dataset(dataset, target_names, **dataviz_args):
    """ Visualization of the dataset given the dictionary of target names
    
    Input:
    dataset: tf.data.Dataset. A tensorflow dataset containing the images to plot
    target_names: dictionary.  The dictionary contains integer keys and string values.
    **dataviz: dictionary.  Contains the arguments necessary to plot the data
        - nrows. int. Total number of rows to display
        - ncols.  int. Total number of columns to display
        - figsize. int. Size of images to plot
        - hspace. int. Vertical space between images
        - wspace. int. Horizontal space between images
        - title_size. int.  Title of the images
       
    Output:
    None: None.
    
    NOTE:  Plot the images in the way you want to show
    """
    # configuration parameters
    figsize = dataviz_args['figsize']
    hspace = dataviz_args['hspace']
    wspace = dataviz_args['wspace']
    nrows = dataviz_args['nrows']
    ncols = dataviz_args['ncols']
    size = dataviz_args['title_size']
    total = nrows*ncols # totalo of images 
    plt.figure(figsize=figsize)
    plt.suptitle('Dataset Visualization', size=size) # change title description
    plt.subplots_adjust(hspace=hspace, wspace=wspace) # adjust plot spacing
    for images, labels in dataset.take(1):   # take 1 image and 1 label randomlhy
        for i in range(total):               # for all images
            img = images[i].numpy().astype("uint8") # transform to a numpy array
            label = int(labels[i])                  # get the label strin into a number
            label = target_names.iloc[label].ClassId  # translate to the correct class
            label_name = target_names.iloc[label].SignName # translate to te correct label name
            plt.subplot(nrows, ncols, i + 1) # prepare to plot
            plt.imshow(img) # plot the image
            plt.title("Label {}\n{}".format(label, label_name)) # print the label and label name
            plt.axis("off") # shut down the axis

In [ ]:
# visualize the dataset
viz_args = dict(nrows=3, ncols=3, figsize=(8,8), hspace=0.7, wspace=1, title_size=20)
visualize_dataset(train_ds, target_names=class_df, **viz_args)

#### Performance of the Dataset

The next step is to ensure we are using correctly the buffer of the PC, Tensorflow provides some functions to have data in memory without affecting the performance and block.

*Dataset.cache()* keeps images in memory after loaded during the first epoch and ensures it will not causes problems on the training process.

*Dataset.prefetch()* makes an overlap of the data preprocessing and model excecution while training

In [ ]:
# autotune the dataset, make a cache of image files, shuffle it and make overlap of the data in memory
AUTOTUNE = tf.data.AUTOTUNE 

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE) 
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

#### Standarization of the data

Data standarization means in the case of images that our images are values between [0, 255], for neural networks is preffered to have the dataset in terms of [0, 1], why? because it helps the model training to act fast.  

*Rescaling* will do the work for us.

In [ ]:
# normalize the data between 0 and 1
normalization_layer = Rescaling(1./255)

In [ ]:
# it will get the first image of the dataset
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]

# Notice the pixels values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))
# Notice that is a batch of images
print(image_batch.shape)
# Notice the shape of the tensor, that is, one image.
print(first_image.shape)

In [ ]:
def define_model(input_shape, num_classes=1, learning_rate=0.001):
    """ Construct the model before train
    
    Input:
    input_shape: tf.Tensor. A tensorflow tensor containing the image in the shape (w,h,3)
    num_classes: int.  Total classes to classify.
    learning_rate: float32.  Hyperparameter for Adam
       
    Output:
    model: keras.engine.sequential.Sequential.  The constructed model and compiled
    """
    model = Sequential([
        Rescaling(1./255, input_shape=input_shape, name='rescaling'),      # rescale the image between [0, 1]
        Conv2D(16, 3, padding='same', activation='relu', name='conv2d_1'), # 16 convs of 3x3 kernels
        MaxPooling2D(name='maxpool2d_1'),                                  # do maxpooling 2x2
        Conv2D(32, 3, padding='same', activation='relu', name='conv2d_2'), # 32 convs of 3x3 kernels
        MaxPooling2D(name='maxpool2d_2'),                                  # do maxpooling 2x2
        Conv2D(64, 3, padding='same', activation='relu', name='conv2d_3'), # 64 convs of 3x3 kernels
        MaxPooling2D(name='maxpool2d_3'),                                  # do maxpooling 2x2
        Flatten(name='flatten'),                                           # flatten
        Dense(128, activation='relu', name='dense'),                       # 128 by relu activation
        Dense(num_classes, name='preds')                                   # total classes will be the softmax output
    ])
    
    optimizer = Adam(learning_rate=learning_rate, name='adam')             # use adam optimizer
    loss = SparseCategoricalCrossentropy(from_logits=True)                 # multiclass classification
    metrics=['accuracy']                                                   # metrics
    
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)         # compile the model
    return model                                                           # return that model

In [ ]:
input_shape = first_image.shape # input shape is (w, h, 3)
num_classes = len(class_df) # calculate the total classes of the dataset, a total of 43
learning_rate = 0.001          # how fast will act the optimizer (Adam)

# make the model
model1 = define_model(input_shape, num_classes, learning_rate)
model1.summary()

#### A better visualization of the model architecture

We provide to you a better way to visualize the model, also it shows you a best way to analize graphically and also include on your paper.

In [ ]:
# YOU MUST INSTALL pydot and GraphViz for this to work
imgpath = './images/naive_model1.png'
plot_model(model1, to_file=imgpath)
#img = load_img(imgpath)
#plt.imshow(img)
#plt.show();

We present here a way to train a model using the type *tf.data.Dataset*, but given straight data you could use the *fit* method in several ways.  I will also like to make callbacks to do early stopping if the model starts to overfit or save the model using the checkpointer, belw is an example.  More information on the [save and load tutorials](https://www.tensorflow.org/tutorials/keras/save_and_load)

    earlystop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)
                                                 
    model.fit(train_images, 
              train_labels,  
              epochs=10,
              validation_data=(test_images, test_labels),
              callbacks=[cp_callback, earlystop]) 

# Include the epoch in the file name (uses `str.format`)
    checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)

    batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
    cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

In [ ]:
def train_model(train_ds, valid_ds, **kwargs):
    """ Trains a model given the datasets of train an validation
    
    Input:
    train_ds: tf.data.Dataset. Instances of training images and labels
    valid_ds: tf.data.Dataset. Instances of validation images and labels
    kwargs: dict.  Contains other parameters of configuration
        - model.  tf.keras.engine.sequential.Sequential.  A model in the sequential format
        - epochs. int. the number of max epocs to train
        - checkpoint_path. string. Path to save the model weights
       
    Output:
    model: tf.keras.callbacks.History.  The result of the training log
    """
    # get the parameters
    model = kwargs['model']
    epochs = kwargs['epochs']
    filepath = kwargs['checkpoint_path']
    save_freq = kwargs['save_freq']
    
    # configure callbacks
    early_stop  = EarlyStopping(monitor='loss', patience=3)
    cp_callback = ModelCheckpoint(filepath=filepath, save_weights_only=True, save_freq=save_freq, verbose=2)
    callbacks = [cp_callback, early_stop]
    
    # train the model
    history = model.fit(train_ds, validation_data=valid_ds, epochs=epochs, callbacks=callbacks)
    return history

In [ ]:
# configure the total epochs to train the model and configure the path to save the weights
epochs = 25
checkpoint_path = '.checkpoint/naive/model'
save_freq = 5*batch_size
train_args = dict(model=model1, epochs=epochs, checkpoint_path=checkpoint_path, save_freq=save_freq)

In [ ]:
history1 = train_model(train_ds, val_ds, **train_args) # train the model

In [ ]:
def plot_results(H, epochs, imgpath=None):
    """ Trains a model given the datasets of train an validation
    
    Input:
    train_ds: tf.data.Dataset. Instances of training images and labels
    valid_ds: tf.data.Dataset. Instances of validation images and labels
    kwargs: dict.  Contains other parameters of configuration
        - model.  tf.keras.engine.sequential.Sequential.  A model in the sequential format
        - epochs. int. the number of max epocs to train
        - checkpoint_path. string. Path to save the model weights
       
    Output:
    model: tf.keras.callbacks.History.  The result of the training log
    """
    
    # accuracy and validation accuracy
    acc = H.history['accuracy']
    val_acc = H.history['val_accuracy']

    # loss and validation loss
    loss = H.history['loss']
    val_loss = H.history['val_loss']

    epochs_range = range(epochs)

    # plot training and validation accuracy and loss, finally save
    plt.figure(figsize=(12, 6))
    plt.suptitle('Model Training Results', size=20)
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.xlabel('Epoch no.')
    plt.ylabel('Accuracy')    
    plt.title('Training and Validation Accuracy')

    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.xlabel('Epoch no.')
    plt.ylabel('Loss')    
    plt.title('Training and Validation Loss')
    plt.savefig(imgpath) if imgpath is not None else _    
    plt.show()

In [ ]:
plot_epoch = len(history1.epoch)
plot_path = './images/training_naive_model1'
plot_results(history1, plot_epoch, plot_path)

#### Inputs and Ouputs format

If you ask, the input and outputs format is very important to make, in this cas a classification.  Bad format will make the code to not run or crash. Below you will see the types expected from the input and the output tensors

In [ ]:
model1.inputs

In [ ]:
model1.output

In [ ]:
model1

#### Plot Prediction Results

Lets plot the prediction results.  As you see above we need an impot tensor of (1, 128, 128, 3), so we have to load the test images from the path, convert to an array and expand its dimentions to the batch to predict, currently only one.

In [ ]:
def plot_prediction(test_df, target_names, model, imgpath=None):
    """ Trains a model given the datasets of train an validation
    
    Input:
    test_df: pd.DataFrame. A test dataframe corresponding to the current images and labels
    target_names: pd.DataFrame. The dataframe corresponding to the current classes
    model: keras.engine.sequential.Sequential.  A model corresponding of the current neural net  
    nimages: int. Number of images to display
    imgpath: 
       
    Output:
    None
    """
    plt.figure(figsize=(8,8))
    ntest = len(test_df)
    rand_ix = np.random.randint(ntest)
    test_path = test_df['Path'][rand_ix]
    test_label = int(test_df['ClassId'][rand_ix]) 
    img = load_img(test_path, target_size=(img_height, img_width))
    img_array = img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch
    pred = model.predict(img_array)
    score = tf.nn.softmax(pred[0])
    acc = np.round(100*np.max(score),2)
    pred_label = np.argmax(score)
    pred_label = target_names.iloc[pred_label].ClassId
    pred_name = target_names.iloc[pred_label].SignName        
        
    plt.title('Label {}\nPrediction {}\n{}\n{}%'.format(test_label, pred_label, pred_name, acc))
    plt.imshow(img);
    plt.axis("off");
    plt.savefig(imgpath) if imgpath is not None else _ 

In [ ]:
def plot_predictions(test_df, target_names, model, imgpath=None):
    """ Trains a model given the datasets of train an validation
    
    Input:
    test_df: pd.DataFrame. A test dataframe corresponding to the current images and labels
    target_names: pd.DataFrame. The dataframe corresponding to the current classes
    model: keras.engine.sequential.Sequential.  A model corresponding of the current neural net  
    nimages: int. Number of images to display
    imgpath: 
       
    Output:
    None
    """
    plt.figure(figsize=(8,8))
    ntest = len(test_df)
    test_labels = []
    pred_labels = []
    pred_acc = []
    for i in range(9):
        rand_ix = np.random.randint(ntest)
        test_path = test_df['Path'][rand_ix]
        test_label = int(test_df['ClassId'][rand_ix]) 
        img = load_img(test_path, target_size=(img_height, img_width))
        img_array = img_to_array(img)
        img_array = tf.expand_dims(img_array, 0) # Create a batch

        pred = model.predict(img_array)
        score = tf.nn.softmax(pred[0])
        acc = np.round(100*np.max(score),2)
        pred_label = np.argmax(score)
        pred_label = target_names.iloc[pred_label].ClassId
        pred_name = target_names.iloc[pred_label].SignName        
        
        test_labels.append(test_label)
        pred_labels.append(pred_label)
        pred_acc.append(acc)
        
        plt.subplot(3, 3, i + 1)
        plt.title('Label {}\nPrediction {}\n{}\n{}%'.format(test_label, pred_label, pred_name, acc), pad=1)
        plt.subplots_adjust(hspace=1, wspace=2)
        plt.imshow(img);
        plt.axis("off");
    plt.savefig(imgpath) if imgpath is not None else _ 
    test_acc = (np.sum(pred_acc)/len(pred_acc))*(np.sum(np.array(test_labels)==np.array(pred_labels))/len(test_labels))
    print('Test accuracy: %.3f%%' % test_acc)

In [ ]:
# plot 'n' predictions
imgpath='./images/predictions_naive_model1'
plot_predictions(test_df=test_df, target_names=class_df, model=model1, imgpath=imgpath)

In [ ]:
def plot_topk_predictions(test_df, target_names, model, nimages=3, k=5, imgpath=None):
    """ Plot top k predictions, will show the input image and the top predictions
    
    Input:
    test_df:      pd.DataFrame. A test dataframe corresponding to the current images and labels
    target_names: pd.DataFrame. The dataframe corresponding to the current classes
    model: keras.engine.sequential.Sequential.  A model corresponding of the current neural net  
    nimages: int. Number of images to plot
    k. int. Predictions of the nimages
    imgpath: str. path of the file to save
       
    Output:
    None
    """
    plt.figure(figsize=(12,12))
    plt.suptitle('Top {} predictions'.format(k), size=20)
    plt.subplots_adjust(hspace=0, wspace=0.2)
    nrows = nimages
    ncols = k + 1
    ntest = len(class_df)
    for row in range(nimages):
        rand_ix = np.random.randint(ntest)
        test_path = test_df['Path'][rand_ix]
        test_label = int(test_df['ClassId'][rand_ix]) 
        img = load_img(test_path, target_size=(img_height, img_width))
        img_array = img_to_array(img)
        img_array = tf.expand_dims(img_array, 0) # Create a batch

        pred = model.predict(img_array)
        score = tf.nn.softmax(pred)
        top5 = tf.nn.top_k(score, k=k)
        
        scores = list(np.round(top5.values[0]*100,2))
        labels = list(top5.indices[0].numpy())      
        test_name =  target_names.iloc[test_label].SignName
        
        curr_row = row*ncols + 1
        plt.subplot(nrows, ncols, curr_row)
        plt.imshow(img) 
        plt.title('Label {}\n{}\n'.format(test_label, test_name, pad=2))
        plt.axis('off')
        for c in range(ncols - 1):
            curr_col = c + curr_row + 1
            plt.subplot(nrows, ncols, curr_col)
            label = labels[c]
            label = target_names.iloc[label].ClassId
            name = target_names.iloc[label].SignName
            score = scores[c]
            label_name = target_names.iloc[label].SignName
            plt.title('{}\n{:.2f}%\n{}'.format(label, score, label_name))
            plt.subplots_adjust(hspace=0.1, wspace=0.5)            
            img = load_img('Meta/' + str(label) + '.png')            
            plt.imshow(img)
            plt.axis('off')
        plt.savefig(imgpath) if imgpath is not None else _ 

In [ ]:
nimages = 3
total_preds = 5
imgpath = './images/topk_preds_naive_model1'
plot_topk_predictions(test_df=test_df, target_names=class_df, model=model1, nimages=nimages, k=total_preds, imgpath=imgpath)

In [ ]:
def prepare_labels(test_df, target_names, model):
    """ Prepare labels for confusion matrix
    
    Input:
    test_df:      pd.DataFrame. A test dataframe corresponding to the current images and labels
    target_names: pd.DataFrame. The dataframe corresponding to the current classes
    model: keras.engine.sequential.Sequential.  A model corresponding of the current neural net  
       
    Output:
    test_labels. list.  A list of the ground truth labels
    test_preds. list. A list of predicted labels
    """
    total = len(test_df)
    nclass = len(target_names)
    test_labels = []
    pred_labels = []
    for i in range(total):
        rand_ix = np.random.randint(nclass)
        test_path = test_df['Path'][rand_ix]
        test_label = int(test_df['ClassId'][rand_ix]) 
        img = load_img(test_path, target_size=(img_height, img_width))
        img_array = img_to_array(img)
        img_array = tf.expand_dims(img_array, 0) # Create a batch

        pred = model.predict(img_array)
        score = tf.nn.softmax(pred[0])
        pred_label = np.argmax(score)
        pred_label = target_names.iloc[pred_label].ClassId
        
        test_labels.append(test_label)
        pred_labels.append(pred_label)
        
    return test_labels, pred_labels

In [ ]:
# get the labels and predictions
y_test, y_pred = prepare_labels(test_df=test_df, target_names=class_df, model=model1)

In [ ]:
# make a confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

In [ ]:
# show unique labels on the test set
np.unique(y_test)

In [ ]:
# show unique labels of predictions
np.unique(y_pred)

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion Matrix', cmap=plt.cm.Blues, imgpath=None):
    """ Plot a pretty confusion matrix
    
    Input:
    cm:  array.  An array composed by a confusion matrix
    classes: list.  A list of classes
    title: str. The title to display of the confusion matrix
    imgpath: str. The path to save the confusion matrix
       
    Output:
    None
    """
    plt.figure(figsize=(12,12))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    ticks = np.arange(len(classes))
    plt.xticks(ticks, classes, rotation=90)
    plt.yticks(ticks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    thresh = cm.max() / 2.0
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i,j], horizontalalignment='center', color='white' if cm[i, j] > thresh else 'black')
    plt.tight_layout()
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.grid(False)
    plt.savefig(imgpath) if imgpath is not None else _ 

In [ ]:
# prepare and plot the confusion matrix
classes = [class_df.iloc[i].SignName for i in np.unique(y_pred)]
title='Confusion Matrix'
imgpath='./images/confusion_matrix_naive_model1'
plot_confusion_matrix(cm, classes=classes, title=title, imgpath=imgpath)

In [ ]:
# print the classification report
print(classification_report(y_test, y_pred))

In [ ]:
# we cannot do horizontal flip because there are several signals that are go-left or go-right for example
data_augmentation = Sequential(
  [      
    RandomRotation(0.1, input_shape=(img_height, img_width, 3)),
    RandomZoom(0.1),
    RandomTranslation(0.1, 0.1),
    RandomContrast(0.1)
  ]
)

In [ ]:
def define_model(input_shape, num_classes=1, learning_rate=0.001):
    """ Construct the model before train
    
    Input:
    input_shape: tf.Tensor. A tensorflow tensor containing the image in the shape (w,h,3)
    num_classes: int.  Total classes to classify.
    learning_rate: float32.  Hyperparameter for Adam
       
    Output:
    model: keras.engine.sequential.Sequential.  The constructed model and compiled
    """
    model = Sequential([
        data_augmentation,                                                 # do data augmentation
        Rescaling(1./255, name='rescaling'),                               # rescale the image between [0, 1]
        Conv2D(16, 3, padding='same', activation='relu', name='conv2d_1'), # 16 convs of 3x3 kernels
        MaxPooling2D(name='maxpool2d_1'),                                  # do maxpooling 2x2
        Conv2D(32, 3, padding='same', activation='relu', name='conv2d_2'), # 32 convs of 3x3 kernels
        MaxPooling2D(name='maxpool2d_2'),                                  # do maxpooling 2x2
        Conv2D(64, 3, padding='same', activation='relu', name='conv2d_3'), # 64 convs of 3x3 kernels
        MaxPooling2D(name='maxpool2d_3'),                                  # do maxpooling 2x2
        Dropout(0.1),                                                      # dropout layer with a 0.1 keep probability
        Flatten(name='flatten'),                                           # flatten
        Dense(128, activation='relu', name='dense'),                       # 128 by relu activation
        Dense(num_classes, name='preds')                                   # total classes will be the softmax output
    ])
    
    optimizer = Adam(learning_rate=learning_rate, name='adam')             # use adam optimizer
    loss = SparseCategoricalCrossentropy(from_logits=True)                 # multiclass classification
    metrics=['accuracy']                                                   # metrics
    
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)         # compile the model
    return model                                                           # return that model

In [ ]:
input_shape = first_image.shape # input shape is (w, h, 3)
num_classes = len(class_df) # calculate the total classes of the dataset, a total of 43
learning_rate = 0.001          # how fast will act the optimizer (Adam)

# make the model
model2 = define_model(input_shape, num_classes, learning_rate)
model2.summary()

In [ ]:
# configure the total epochs to train the model and configure the path to save the weights
epochs = 25
checkpoint_path = '.checkpoint/aug_dropout/model'
save_freq = 5*batch_size
train_args = dict(model=model2, epochs=epochs, checkpoint_path=checkpoint_path, save_freq=save_freq)
history2 = train_model(train_ds, val_ds, **train_args) # train the model

In [ ]:
plot_epoch = len(history2.epoch)
plot_path = './images/aug_dropout_model'
plot_results(history2, plot_epoch, plot_path)

In [ ]:
first_image.shape

In [ ]:
# plot 9 predictions
imgpath='./images/aug_dropout_preds'
plot_predictions(test_df=test_df, target_names=class_df, model=model2, imgpath=imgpath)

In [ ]:
nimages = 5
total_preds = 5
imgpath = './images/topk_preds_aug_dropout'
plot_topk_predictions(test_df=test_df, target_names=class_df, model=model2, nimages=nimages, k=total_preds, imgpath=imgpath)

In [ ]:
# this part will take a long time, please wait!

# prepare the confusion matrix
y_test, y_pred = prepare_labels(test_df=test_df, target_names=class_df, model=model2)
cm = confusion_matrix(y_test, y_pred)

# prepare and plot the confusion matrix
classes = [class_df.iloc[i].SignName for i in np.unique(y_pred)]
title='Confusion Matrix'
imgpath='./images/confusion_matrix_aug_dropout'
plot_confusion_matrix(cm, classes=classes, title=title, imgpath=imgpath)

In [ ]:
# print the classification report
print(classification_report(y_test, y_pred))

In [ ]:
# rescale pipeline
rescale = Rescaling(1./255)

In [ ]:
global_average_layer = GlobalAveragePooling2D()

In [ ]:
# Create the base model from the pre-trained model MobileNet V2

input_shape = tuple(first_image.shape)
base_model = ResNet50V2(input_tensor=Input(shape=input_shape), include_top=False, weights='imagenet')

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
base_model.summary()

In [ ]:
def define_model(input_shape, num_classes=1, learning_rate=0.001):
    """ Construct the model before train
    
    Input:
    input_shape: tf.Tensor. A tensorflow tensor containing the image in the shape (w,h,3)
    num_classes: int.  Total classes to classify.
    learning_rate: float32.  Hyperparameter for Adam
       
    Output:
    model: keras.engine.sequential.Sequential.  The constructed model and compiled
    """
    inputs = Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = rescale(inputs)
    x = base_model(x, training=False)
    x = global_average_layer(x)
    x = Dropout(0.1)(x)
    x = Flatten()(x)
    x = Dense(128)(x)
    outputs = Dense(num_classes)(x)
    model = Model(inputs, outputs)

    optimizer = Adam(learning_rate=learning_rate)
    loss = SparseCategoricalCrossentropy(from_logits=True)
    metrics=['accuracy']

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
    
    return model

In [ ]:
input_shape = first_image.shape # input shape is (w, h, 3)
num_classes = len(class_df) # calculate the total classes of the dataset, a total of 43
learning_rate = 0.001          # how fast will act the optimizer (Adam)

# make the model
model3 = define_model(input_shape, num_classes, learning_rate)
model3.summary()

In [ ]:
# configure the total epochs to train the model and configure the path to save the weights
epochs = 25
checkpoint_path = '.checkpoint/transfer_learning/model'
save_freq = 5*batch_size
train_args = dict(model=model3, epochs=epochs, checkpoint_path=checkpoint_path, save_freq=save_freq)
history3 = train_model(train_ds, val_ds, **train_args) # train the model

In [ ]:
plot_epoch = len(history3.epoch)
plot_path = './images/transfer_learning'
plot_results(history3, plot_epoch, plot_path)

In [ ]:
# plot 'n' predictions
imgpath='./images/predictions_transfer_learning'
plot_predictions(test_df=test_df, target_names=class_df, model=model3, imgpath=imgpath)

In [ ]:
nimages = 6
total_preds = 4
imgpath = './images/topk_preds_transfer_learning'
plot_topk_predictions(test_df=test_df, target_names=class_df, model=model3, nimages=nimages, k=total_preds, imgpath=imgpath)

In [ ]:
# this part will take a long time, please wait!

# prepare the confusion matrix
y_test, y_pred = prepare_labels(test_df=test_df, target_names=class_df, model=model3)
cm = confusion_matrix(y_test, y_pred)

# prepare and plot the confusion matrix
classes = [class_df.iloc[i].SignName for i in np.unique(y_pred)]
title='Confusion Matrix'
imgpath='./images/confusion_matrix_aug_dropout'
plot_confusion_matrix(cm, classes=classes, title=title, imgpath=imgpath)

In [ ]:
# print the classification report
print(classification_report(y_test, y_pred))

#### Saving, Loading the Model and Predict

In [ ]:
# Model and weights path
checkpoint_dir = '.checkpoint/best/model'
# Save the model
model1.save(checkpoint_dir)
# Save the weights
model1.save_weights(checkpoint_dir)

In [ ]:
def define_model(input_shape, num_classes=1, learning_rate=0.001):
    """ Construct the model before train
    
    Input:
    input_shape: tf.Tensor. A tensorflow tensor containing the image in the shape (w,h,3)
    num_classes: int.  Total classes to classify.
    learning_rate: float32.  Hyperparameter for Adam
       
    Output:
    model: keras.engine.sequential.Sequential.  The constructed model and compiled
    """
    model = Sequential([
        Rescaling(1./255, input_shape=input_shape, name='rescaling'),      # rescale the image between [0, 1]
        Conv2D(16, 3, padding='same', activation='relu', name='conv2d_1'), # 16 convs of 3x3 kernels
        MaxPooling2D(name='maxpool2d_1'),                                  # do maxpooling 2x2
        Conv2D(32, 3, padding='same', activation='relu', name='conv2d_2'), # 32 convs of 3x3 kernels
        MaxPooling2D(name='maxpool2d_2'),                                  # do maxpooling 2x2
        Conv2D(64, 3, padding='same', activation='relu', name='conv2d_3'), # 64 convs of 3x3 kernels
        MaxPooling2D(name='maxpool2d_3'),                                  # do maxpooling 2x2
        Flatten(name='flatten'),                                           # flatten
        Dense(128, activation='relu', name='dense'),                       # 128 by relu activation
        Dense(num_classes, name='preds')                                   # total classes will be the softmax output
    ])
    
    optimizer = Adam(learning_rate=learning_rate, name='adam')             # use adam optimizer
    loss = SparseCategoricalCrossentropy(from_logits=True)                 # multiclass classification
    metrics=['accuracy']                                                   # metrics
    
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)         # compile the model
    return model                                                           # return that model

In [ ]:
# Construct and define the same model architecture
# Load the weights
# predict
best_model = define_model(input_shape, num_classes, learning_rate=learning_rate)

In [ ]:
best_model.load_weights(checkpoint_dir)
plot_prediction(test_df=test_df, target_names=class_df, model=best_model, imgpath=None)

In [ ]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = ".checkpoint/best/model-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
# get the latest checkpoint
latest = tf.train.latest_checkpoint(checkpoint_dir)
# Restore the weights
best_model.load_weights(latest)

In [ ]:
plot_prediction(test_df=test_df, target_names=class_df, model=best_model, imgpath=None)

#### Convert a Saved Model to TFLite

In [ ]:
def representative_dataset():
    for _ in range(100):
        data = np.random.rand(1, 128, 128, 3)
        yield [data.astype(np.float32)]

In [ ]:
def convert_tflite(checkpoint_path, tflite_name, support_type=None):
    # Convert the model
    converter = tf.lite.TFLiteConverter.from_saved_model(checkpoint_path) # path to the SavedModel directory
    
    if support_type == 'float':
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.representative_dataset = representative_dataset 
    if support_type == 'float16':
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.target_spec.supported_types = [tf.float16]
    if support_type == 'int8':
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.representative_dataset = representative_dataset        
        converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
        converter.inference_input_type = tf.int8  # or tf.uint8
        converter.inference_output_type = tf.int8  # or tf.uint8
    if support_type == 'int16x8a':
        converter.representative_dataset = representative_dataset
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.target_spec.supported_ops = [tf.lite.OpsSet.EXPERIMENTAL_TFLITE_BUILTINS_ACTIVATIONS_INT16_WEIGHTS_INT8]
    if support_type == 'int16x8b':
        converter.representative_dataset = representative_dataset
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        converter.target_spec.supported_ops = [tf.lite.OpsSet.EXPERIMENTAL_TFLITE_BUILTINS_ACTIVATIONS_INT16_WEIGHTS_INT8, 
                                               tf.lite.OpsSet.TFLITE_BUILTINS]
    
    tflite_model = converter.convert()

    # Save the model
    tflite_path = os.path.join(checkpoint_path, tflite_name)
    with open(tflite_path, 'wb') as f:
        f.write(tflite_model)

In [ ]:
checkpoint_path = '.checkpoint/best/model'
tflite_name = 'model.tflite'

convert_tflite(checkpoint_path, tflite_name=tflite_name, support_type='int8')

#### Load and use the model

In [ ]:
def load_tflite_model(checkpoint_path, tflite_name):
    # Load the TFLite model in TFLite Interpreter
    tflite_path = os.path.join(checkpoint_path, tflite_name)
    interpreter = tf.lite.Interpreter(tflite_path)
    
    # Get input and output tensors.
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.allocate_tensors()
    # input details
    print(input_details)
    #print()
    # output details
    print(output_details)
    return interpreter

In [ ]:
tflite_model = load_tflite_model(checkpoint_path, tflite_name)

In [ ]:
def tflite_predict(test_df, interpreter):
    rand_ix = np.random.randint(len(test_df))
    test_path = test_df['Path'][rand_ix]
    test_label = int(test_df['ClassId'][rand_ix]) 
    img = load_img(test_path, target_size=(img_height, img_width))
    img_array = img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch
    interpreter.set_tensor(input_details[0]['index'], img_array)
    # run the inference
    interpreter.invoke()
    
    # output_details[0]['index'] = the index which provides the input
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    #print("the output is {}".format(output_data))
    score = tf.nn.softmax(output_data)
    #print("the softmax activation is {}".format(score))
    
    #top_k = tf.nn.top_k(score, k=5)
    #top_k.indices
    #print('The top_k indices are {}'.format(top_k.indices))
    acc = np.round(100*np.max(score),2)
    #print('The acc is {}'.format(acc))
    #acc_top_k = np.round(100*np.max(top_k),2)
    #print('The top k is {}'.format(acc_top_k))    
    pred_label = np.argmax(score)
    pred_label = class_df.iloc[pred_label].ClassId
    pred_name = class_df.iloc[pred_label].SignName
    #print('The pred label is {}'.format(pred_label))
    #print('The pred name is {}'.format(pred_name))
    plt.title('Label {}\nPrediction {}\n{}\n{}%'.format(test_label, pred_label, pred_name, acc))
    plt.imshow(img);
    plt.axis("off");
    plt.savefig(imgpath) if imgpath is not None else _ 
    return pred_label, pred_name, acc

In [ ]:
tflite_predict(test_df, interpreter)